In [1]:
# import modules
from datasets import load_dataset
from transformers import LlamaForCausalLM, LlamaTokenizer
import argparse
import socket
from dataset import *
import torch
import os
import json
import pickle

/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load data
# data = load_dataset("json", data_files=./data)

tokenizer = LlamaTokenizer.from_pretrained(
    './llama',
    add_eos_token = True # add end-of-sentence token
)
tokenizer.pad_token_id = 0


def generate_prompt(data_point):
    return data_point["context"]

def tokenize(prompt,tokenizer, CUTOFF_LEN):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN + 1,
        padding="max_length",
    )
    return {
        "input_ids": result["input_ids"][:-1],
        "attention_mask": result["attention_mask"][:-1],
    }


def generate_and_tokenize_prompt(data_point, tokenizer, CUTOFF_LEN):
    prompt = generate_prompt(data_point)
    return tokenize(prompt, tokenizer, CUTOFF_LEN)



In [9]:
case_hold = build_dataset(['case_hold'], './dataset')[0]
dataset = case_hold

100%|██████████| 1/1 [00:02<00:00,  2.58s/it]

Datasets are succesfully loaded


In [10]:
#dataset.select_columns(['context', 'endings'])
CUTOFF_LEN = 512
tokenizer = LlamaTokenizer.from_pretrained(
    "./llama", add_eos_token=True
)
tokenizer.pad_token_id = 0
def generate_prompt(data_point):
    return data_point["context"]


def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN + 1,
        padding="max_length",
    )
    return {
        "input_ids": result["input_ids"][:-1],
        "attention_mask": result["attention_mask"][:-1],
    }


def generate_and_tokenize_prompt(data_point):
    prompt = generate_prompt(data_point)
    return tokenize(prompt)

train_dataest = dataset['train'].shuffle().map(generate_and_tokenize_prompt)

Map:   8%|▊         | 3765/45000 [00:05<00:55, 740.53 examples/s]

In [21]:
def generate_prompt(data_point):
    return data_point["input"]


def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN + 1,
        padding="max_length",
    )
    return {
        "input_ids": result["input_ids"][:-1],
        "attention_mask": result["attention_mask"][:-1],
    }


def generate_and_tokenize_prompt(data_point):
    prompt = generate_prompt(data_point)
    return tokenize(prompt)

In [29]:
train_data

Dataset({
    features: ['topic', 'input', 'input_ids', 'attention_mask'],
    num_rows: 52456
})

In [16]:
CUTOFF_LEN = 512
quora_dataset = load_dataset('json', data_files = './data/quora_chat_data.json')
data = quora_dataset
tokenizer = LlamaTokenizer.from_pretrained(
    "./llama", add_eos_token=True
)
tokenizer.pad_token_id = 0
VAL_SET_SIZE = 2000
if VAL_SET_SIZE > 0:
    train_val = data["train"].train_test_split(
        test_size=VAL_SET_SIZE, shuffle=True, seed=42
    )
    train_data = train_val["train"].shuffle().map(generate_and_tokenize_prompt)
    val_data = train_val["test"].shuffle().map(generate_and_tokenize_prompt)
else:
    train_data = data["train"].shuffle().map(generate_and_tokenize_prompt)
    val_data = None

Loading cached split indices for dataset at /home/laal_intern003/.cache/huggingface/datasets/json/default-ea43db81aff75e51/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-d694195a3de92917.arrow and /home/laal_intern003/.cache/huggingface/datasets/json/default-ea43db81aff75e51/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-523ec231a7ab784c.arrow


In [20]:
train_data['topic']

['Is it dangerous to have a resting heart rate of 88 bpm? How can I lower it?',
 'What are the best third-party spell checkers for myMail, Gmail and Outlook on my mobile?',
 "What'd be the maximum wavelength of electromagnetic radiation we'd use to read a message in a paper?",
 'Do employees at Vector Group have a good work-life balance? Does this differ across positions and departments?',
 'Why do some people on QUORA ask questions that they can easily findout on Google?',
 'Which application is this? These are apps used by teachers to write down things and explain topics to students. I want the name of this application.',
 'How can I find out what the total cost of living in Palo Alto for a year for a single 20-something would be?',
 'Do black holes exist?',
 "What is Billy Costa's mailing address?",
 'How shall I prepare for CA final Nov 16 exams',
 'What are some of the best beaches in Goa?',
 'How do I make money as a teen online?',
 'What are jobs suited for good observer?',
 'Re

In [2]:
case_hold = build_dataset(['case_hold'], './dataset')[0]

NameError: name 'build_dataset' is not defined

In [15]:
case_hold

DatasetDict({
    train: Dataset({
        features: ['context', 'endings', 'label'],
        num_rows: 45000
    })
    test: Dataset({
        features: ['context', 'endings', 'label'],
        num_rows: 3600
    })
    validation: Dataset({
        features: ['context', 'endings', 'label'],
        num_rows: 3900
    })
})

In [3]:
case_hold['train']['endings'][0][case_hold['train']['label'][0]]

'holding that possession of a pipe bomb is a crime of violence for purposes of 18 usc  3142f1'

In [4]:
import string

def fill(context, ending):
    masked = '(<HOLDING>)'
    context.replace(masked, ending)
    return context


In [19]:
masked = case_hold['train']['endings'][0][case_hold['train']['label'][0]]
case_hold['train']['context'][0].replace('(<HOLDING>)', masked)

"Drapeau’s cohorts, the cohort would be a “victim” of making the bomb. Further, firebombs are inherently dangerous. There is no peaceful purpose for making a bomb. Felony offenses that involve explosives qualify as “violent crimes” for purposes of enhancing the sentences of career offenders. See 18 U.S.C. § 924(e)(2)(B)(ii) (defining a “violent felony” as: “any crime punishable by imprisonment for a term exceeding one year ... that ... involves use of explosives”). Courts have found possession of a'bomb to be a crime of violence based on the lack of a nonviolent purpose for a bomb and the fact that, by its very nature, there is a substantial risk that the bomb would be used against the person or property of another. See United States v. Newman, 125 F.3d 863 (10th Cir.1997) (unpublished) holding that possession of a pipe bomb is a crime of violence for purposes of 18 usc  3142f1; United States v. Dodge, 846 F.Supp. 181,"

In [23]:
def answer(data_point):
    return data_point['context'], data_point['endings'][data_point['label']]

def fill_masked(data_point):
    text, masked = answer(data_point)
    return {
        "text" : text.replace('(<HOLDING>)', masked)
    }

In [38]:
case_hold

DatasetDict({
    train: Dataset({
        features: ['context', 'endings', 'label'],
        num_rows: 45000
    })
    test: Dataset({
        features: ['context', 'endings', 'label'],
        num_rows: 3600
    })
    validation: Dataset({
        features: ['context', 'endings', 'label'],
        num_rows: 3900
    })
})

In [39]:
from dataset import *

In [36]:
import datasets
datasetdict = datasets.DatasetDict(
    {
        'test': ds
    }
)

In [46]:
def _answer(data_point):
    return data_point['context'], data_point['endings'][data_point['label']]

def _fill_masked(data_point):
    text, masked = _answer(data_point)
    return {
        "text" : text.replace('(<HOLDING>)', masked)
    }

def rebuild_dataset(dataset):
    datasetdict = {}
    for split in tqdm(['train', 'validation', 'test']):
        datasetdict[split] = dataset[split].map(_fill_masked).select_columns(['text'])
    return datasets.DatasetDict(datasetdict)
    
d = rebuild_dataset(case_hold)

  0%|          | 0/3 [00:00<?, ?it/s]Loading cached processed dataset at /home/laal_intern003/.cache/huggingface/datasets/lex_glue/case_hold/1.0.0/8a66420941bf6e77a7ddd4da4d3bfb7ba88ef48c1d55302a568ac650a095ca3a/cache-9e1fba865d7d682f.arrow
Loading cached processed dataset at /home/laal_intern003/.cache/huggingface/datasets/lex_glue/case_hold/1.0.0/8a66420941bf6e77a7ddd4da4d3bfb7ba88ef48c1d55302a568ac650a095ca3a/cache-ae14c54dfb3f2eda.arrow
Loading cached processed dataset at /home/laal_intern003/.cache/huggingface/datasets/lex_glue/case_hold/1.0.0/8a66420941bf6e77a7ddd4da4d3bfb7ba88ef48c1d55302a568ac650a095ca3a/cache-12846bf084281f0b.arrow
100%|██████████| 3/3 [00:00<00:00, 143.72it/s]


In [44]:
d

DatasetDict({
    train: Dataset({
        features: ['context', 'endings', 'label', 'text'],
        num_rows: 45000
    })
    validation: Dataset({
        features: ['context', 'endings', 'label', 'text'],
        num_rows: 3900
    })
    test: Dataset({
        features: ['context', 'endings', 'label', 'text'],
        num_rows: 3600
    })
})

In [29]:
ds = case_hold['test'].map(fill_masked)

Loading cached processed dataset at /home/laal_intern003/.cache/huggingface/datasets/lex_glue/case_hold/1.0.0/8a66420941bf6e77a7ddd4da4d3bfb7ba88ef48c1d55302a568ac650a095ca3a/cache-49f95a6fe849df2c.arrow


In [31]:
ds['text'][0]

'v. Power Auth. of the State of New York, 81 N.Y.2d 649, 652, 602 N.Y.S.2d 588, 621 N.E.2d 1195, 1196 (N.Y.1993) (same). These cases teach that fair market value is not to be determined in a rarefied realm of abstract calculation, but from the perspective of a hypothetical buyer in the real world — even if her perception of value is based partially on misinformation or (arguably) exaggerated fears about uncertain future events. Other courts, in the same or similar circumstances, have held that the anticipation of increased interference or potential hazards on the land remaining after a taking, which one court has labeled “stigma damages,” can be considered in assessing severance damages. See, e.g., Vector Pipeline, L.P., v. 68.55 Acres of Land, 157 F.Supp.2d 949, 957 (N.D.Ill.2001) holding that stigma damages were properly included in calculation of severance damages where proximity of gas pipeline could trigger fears about possible mishaps; United States v. 14.38 Acres of Land, 80 F.3

In [20]:
case_hold['train']['context'][0]

"Drapeau’s cohorts, the cohort would be a “victim” of making the bomb. Further, firebombs are inherently dangerous. There is no peaceful purpose for making a bomb. Felony offenses that involve explosives qualify as “violent crimes” for purposes of enhancing the sentences of career offenders. See 18 U.S.C. § 924(e)(2)(B)(ii) (defining a “violent felony” as: “any crime punishable by imprisonment for a term exceeding one year ... that ... involves use of explosives”). Courts have found possession of a'bomb to be a crime of violence based on the lack of a nonviolent purpose for a bomb and the fact that, by its very nature, there is a substantial risk that the bomb would be used against the person or property of another. See United States v. Newman, 125 F.3d 863 (10th Cir.1997) (unpublished) (<HOLDING>); United States v. Dodge, 846 F.Supp. 181,"

In [7]:
case_hold['train']['context'][0] == fill(case_hold['train']['context'][0], case_hold['train']['endings'][0][case_hold['train']['label'][0]])

True

In [24]:
for i in range(10):
    answer = case_hold['train']['endings'][i][case_hold['train']['label'][i]]
    case_hold['train']['context'][i] = fill(case_hold['train']['context'][i], answer)

In [25]:
case_hold['train']['context'][0]

"Drapeau’s cohorts, the cohort would be a “victim” of making the bomb. Further, firebombs are inherently dangerous. There is no peaceful purpose for making a bomb. Felony offenses that involve explosives qualify as “violent crimes” for purposes of enhancing the sentences of career offenders. See 18 U.S.C. § 924(e)(2)(B)(ii) (defining a “violent felony” as: “any crime punishable by imprisonment for a term exceeding one year ... that ... involves use of explosives”). Courts have found possession of a'bomb to be a crime of violence based on the lack of a nonviolent purpose for a bomb and the fact that, by its very nature, there is a substantial risk that the bomb would be used against the person or property of another. See United States v. Newman, 125 F.3d 863 (10th Cir.1997) (unpublished) (<HOLDING>); United States v. Dodge, 846 F.Supp. 181,"

In [6]:
dataset = build_dataset(['case_hold'], './dataset')[0]

100%|██████████| 1/1 [00:03<00:00,  3.47s/it]

Datasets are succesfully loaded


In [13]:
dataset['test']['context'][0].find('(<HOLDING>)')

'v. Power Auth. of the State of New York, 81 N.Y.2d 649, 652, 602 N.Y.S.2d 588, 621 N.E.2d 1195, 1196 (N.Y.1993) (same). These cases teach that fair market value is not to be determined in a rarefied realm of abstract calculation, but from the perspective of a hypothetical buyer in the real world — even if her perception of value is based partially on misinformation or (arguably) exaggerated fears about uncertain future events. Other courts, in the same or similar circumstances, have held that the anticipation of increased interference or potential hazards on the land remaining after a taking, which one court has labeled “stigma damages,” can be considered in assessing severance damages. See, e.g., Vector Pipeline, L.P., v. 68.55 Acres of Land, 157 F.Supp.2d 949, 957 (N.D.Ill.2001) (<HOLDING>); United States v. 14.38 Acres of Land, 80 F.3d'

In [1]:
import random
def prompt_engineering(data_point):

    random.seed(7)
    prompt_cands = [
    "Please select the most suitable summary of the legal ruling that accompanies the relevant referenced decisions for the specific case. The following excerpt is from the court's decision.",
    "Kindly choose the concise summary of the legal ruling that accompanies the relevant referenced decisions applicable to the given case. Provided below is an excerpt from the court decision.",
    "Please decide on the most appropriate summary of the legal ruling that accompanies the relevant referenced decisions, which are relevant to the given case. Here is an excerpt from the court decision for your consideration.",
    "Here is an excerpt from the court decision for the case. Please choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Consider the following excerpt from the court decision for the case. Your task is to select the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Given the excerpt from the court decision for the case, your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Please refer to the following excerpt from the court decision for the case. Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case, using the excerpt from the court decision provided below.",
    "Please review the following excerpt from the court decision for the case. Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Here is the excerpt from the court decision for the case. Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case."
    ]

    
    _slice = data_point.find('(<HOLDING>)')
    data_point = data_point[:_slice]
    prompt = random.choice(prompt_cands)
    input = prompt + f'\nExcerpt: {data_point}'

    return input

print(prompt_engineering(dataset['test']['context'][0]))

NameError: name 'dataset' is not defined

In [17]:
dataset['test']['endings'][0]

['holding that actual damages under respa included emotional damages',
 'holding that increased fear of flooding was a matter that factfinder could properly consider in assessing severance damages',
 'holding that local buyers fear of possible knapweed infestation legitimately affected calculation of severance damages',
 'recognizing concept of stigma damages as they apply to property which is in fact contaminated',
 'holding that stigma damages were properly included in calculation of severance damages where proximity of gas pipeline could trigger fears about possible mishaps']

In [19]:
[str(n) + f': {c}' for n,c in enumerate(dataset['test']['endings'][0])]

['0: holding that actual damages under respa included emotional damages',
 '1: holding that increased fear of flooding was a matter that factfinder could properly consider in assessing severance damages',
 '2: holding that local buyers fear of possible knapweed infestation legitimately affected calculation of severance damages',
 '3: recognizing concept of stigma damages as they apply to property which is in fact contaminated',
 '4: holding that stigma damages were properly included in calculation of severance damages where proximity of gas pipeline could trigger fears about possible mishaps']

In [21]:
dataset

DatasetDict({
    train: Dataset({
        features: ['context', 'endings', 'label'],
        num_rows: 45000
    })
    test: Dataset({
        features: ['context', 'endings', 'label'],
        num_rows: 3600
    })
    validation: Dataset({
        features: ['context', 'endings', 'label'],
        num_rows: 3900
    })
})

In [1]:
from test import make_dataset, prompt_engineering


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so
/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA SETUP: Loading binary /home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/laal_intern003/anaconda/envs/legal-master/lib/python3.8/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [5]:
import argparse
import random
parser = argparse.ArgumentParser()
parser.add_argument('--data_dir',
                    type = str,
                    default = './dataset',
                    help = 'Where LexGlue dataset is stored')

args = parser.parse_args(args = [])
dataset = make_dataset(args)

Datasets are succesfully loaded


Map:   0%|          | 0/3600 [00:00<?, ? examples/s]

In [34]:

def make_dataset(args):
    # build dataset
    path = args.data_dir

    # load file
    if os.path.isfile(os.path.join(path, 'prompt.pkl')):
        with open(os.path.join(path, 'prompt.pkl'), 'rb') as f:
            dataset = pickle.load(f)
    else:
        dataset_list = build_dataset(['case_hold'], path)
        dataset = dataset_list[0]['test']
        random.seed(7)
        dataset = dataset.shuffle().map(prompt_engineering).select_columns(['input', 'label'])
        
        # save file
        with open(os.path.join(path, 'prompt.pkl'), 'wb') as f:
            pickle.dump(dataset, f)

    return dataset

def prompt_engineering(data_point):

    prompt_cands = [
    "Please select the most suitable summary of the legal ruling that accompanies the relevant referenced decisions for the specific case. The following excerpt is from the court's decision.",
    "Kindly choose the concise summary of the legal ruling that accompanies the relevant referenced decisions applicable to the given case. Provided below is an excerpt from the court decision.",
    "Please decide on the most appropriate summary of the legal ruling that accompanies the relevant referenced decisions, which are relevant to the given case. Here is an excerpt from the court decision for your consideration.",
    "Here is an excerpt from the court decision for the case. Please choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Consider the following excerpt from the court decision for the case. Your task is to select the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Given the excerpt from the court decision for the case, your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Please refer to the following excerpt from the court decision for the case. Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case, using the excerpt from the court decision provided below.",
    "Please review the following excerpt from the court decision for the case. Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Here is the excerpt from the court decision for the case. Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case."
    ] # generated by Chat-GPT

    # print(data_point)    
    _slice = data_point['context'].find('(<HOLDING>)')
    excerpt = data_point['context'][:_slice]
    choices = [str(n) + f': {c}' for n,c in enumerate(data_point['endings'])]
    prompt = random.choice(prompt_cands)
    input = prompt + f'\nExcerpt: {excerpt}' + f'\nChoices: {choices}'

    return {
        'question' : input
    }

In [32]:
import random
random.seed(7)
dataset = dataset.shuffle().map(prompt_engineering).select_columns(['question', 'label'])

Map:   0%|          | 0/3600 [00:00<?, ? examples/s]

KeyError: 'context'

In [31]:
dataset['question'][10]

"Given the excerpt from the court decision for the case, your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.\nExcerpt: of acquittal were entered on these counts by the Court because the jury’s findings did not identify specified unlawful activities from which the funds were acquired by the defendant that were distinct from the alleged money laundering activities. See United States v. Howard, No. 02-079, slip. op. at 8 (D.D.C. Feb. 5, 2003); see also United States v. Seward, 272 F.3d 831, 836 (7th Cir.2001) (“The transaction or transactions that created the criminally-derived proceeds must be distinct from the money-laundering transaction because the money laundering statutes criminalize ‘transaction^] in proceeds, not the transaction^] that create [ ] the proceeds.' ”) (quoting United States v. Mankarious, 151 F.3d 694, 705 (7th Cir.1998)); United States v. Butler, 211 F.3d 826, 830 (4th Cir.2000)

In [1]:

# pipeeline

def make_dataset(args):
    # build dataset
    path = args.data_dir

    # load file
    if os.path.isfile(os.path.join(path, 'prompt.pkl')):
        with open(os.path.join(path, 'prompt.pkl'), 'rb') as f:
            dataset = pickle.load(f)
    else:
        dataset_list = build_dataset(['case_hold'], path)
        dataset = dataset_list[0]['test']
        random.seed(7)
        dataset = dataset.shuffle().map(prompt_engineering).select_columns(['question', 'label'])
        
        # save file
        with open(os.path.join(path, 'prompt.pkl'), 'wb') as f:
            pickle.dump(dataset, f)

    return dataset

def prompt_engineering(data_point):

    prompt_cands = [
    "Please select the most suitable summary of the legal ruling that accompanies the relevant referenced decisions for the specific case. The following excerpt is from the court's decision.",
    "Kindly choose the concise summary of the legal ruling that accompanies the relevant referenced decisions applicable to the given case. Provided below is an excerpt from the court decision.",
    "Please decide on the most appropriate summary of the legal ruling that accompanies the relevant referenced decisions, which are relevant to the given case. Here is an excerpt from the court decision for your consideration.",
    "Here is an excerpt from the court decision for the case. Please choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Consider the following excerpt from the court decision for the case. Your task is to select the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Given the excerpt from the court decision for the case, your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Please refer to the following excerpt from the court decision for the case. Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case, using the excerpt from the court decision provided below.",
    "Please review the following excerpt from the court decision for the case. Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.",
    "Here is the excerpt from the court decision for the case. Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case."
    ] # generated by Chat-GPT

    
    _slice = data_point['context'].find('(<HOLDING>)')
    excerpt = data_point['context'][:_slice]
    choices = [str(n) + f': {c}' for n,c in enumerate(data_point['endings'])]
    prompt = random.choice(prompt_cands)
    input = prompt + f'\nExcerpt: {excerpt}' + f'\nChoices: {choices}'

    return {
        'question' : input
    }

In [2]:
from dataset import *
dataset = make_dataset(args)

NameError: name 'args' is not defined

In [13]:
dataset['question'][10]

"Please review the following excerpt from the court decision for the case. Your task is to choose the most appropriate short summary of the legal ruling that accompanies the referenced decisions relevant to the case.\nExcerpt: been very emphatic that, except in cases where only one inference can be drawn from the facts, negligence, proximate cause, and foreseeability are questions of fact for the jury. See St. Clair v. Denny, 245 Kan. 414, 781 P.2d 1043, 1047 (1989); Baker v. City of Garden City, 240 Kan. 554, 731 P.2d 278, 281 (1987); Gard, 400 P.2d at 1000, 1002; Rowell, 176 P.2d at 595. As might be expected in an area of the law in which so much depends on the factual scenario of the particular case, the holdings of the Kansas cases that address causation are not clearly dispositive of the issue. However, we find that those cases whose facts most closely approximate those of the instant case support the conclusion that the district court improperly granted summary judgment. See Stee